In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from ydata_profiling import ProfileReport
from autogluon.tabular import TabularPredictor,TabularDataset
import os

In [3]:
model_save_path = 'D:\ArcgisData\pred_soildeep\pred_model'

In [5]:
# 读取数据
file_path = r"D:\ArcgisData\pred_soildeep\feature_result\feature_soildeep.csv"
data = pd.read_csv(file_path)
data


,lon,lat,deep,AP,PRE,TMP,SOILQS,NIGHT
0,107.825330,26.105950,120.0,9296,868.6667,174.25000,12,0.40
1,108.035830,26.111340,120.0,9338,882.4167,167.66667,11,0.29
2,107.821020,26.117300,123.0,9297,868.0833,174.75000,12,0.37
3,107.890680,26.124260,120.0,9327,870.6667,172.25000,11,0.32
4,107.818020,26.156750,122.0,9184,875.5000,168.83333,12,0.51
...,...,...,...,...,...,...,...,...
7589,106.977203,26.506019,130.0,8859,830.0833,150.83333,11,1.00
7590,106.948998,26.496857,135.0,8891,826.8333,152.75000,11,1.06
7591,106.975952,26.508483,135.0,8866,830.0833,150.83333,11,1.00
7592,106.949975,26.467226,130.0,8918,829.5833,153.50000,11,6.45


In [8]:
data.drop_duplicates(inplace=True)
data.describe()

,lon,lat,deep,AP,PRE,TMP,SOILQS,NIGHT
count,7585.000000,7585.000000,7585.000000,7585.000000,7585.000000,7585.000000,7585.000000,7585.000000
mean,106.202000,26.810610,134.157811,8733.603691,857.217425,149.830367,11.705471,0.744658
std,1.386224,0.779634,16.920585,458.683528,51.606031,18.149287,0.987930,1.331370
min,103.637677,24.768798,40.000000,7817.000000,717.750000,89.250000,11.000000,0.000000
25%,104.958428,26.313360,120.000000,8321.000000,821.916700,138.666670,11.000000,0.320000
50%,106.173320,26.817611,135.000000,8769.000000,849.000000,152.916670,11.000000,0.400000
75%,107.209937,27.289147,150.000000,9079.000000,892.166700,162.416670,12.000000,0.610000
max,109.469500,29.028264,162.000000,9739.000000,1048.083400,206.500000,15.000000,26.220000


In [9]:
data = data[['lon','lat','deep','AP','PRE','TMP','SOILQS','NIGHT']]
data['deep'] = data['deep'].astype('float64')
data['AP'] = data['AP'].astype('float64')
data.dtypes

lon       float64
lat       float64
deep      float64
AP        float64
PRE       float64
TMP       float64
SOILQS      int64
NIGHT     float64
dtype: object

In [10]:
train_data, test_data = train_test_split(data,test_size=0.2,random_state=42)

In [11]:
label = 'deep'

In [12]:
predictor = TabularPredictor(label=label,eval_metric='r2',path=os.path.join(model_save_path,"test0901"),problem_type='regression')
predictor.fit(train_data,tuning_data=test_data,time_limit=10800)

Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "D:\ArcgisData\pred_soildeep\pred_model\test0901\"
AutoGluon Version:  0.8.2
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   416.86 GB / 808.56 GB (51.6%)
Train Data Rows:    6068
Train Data Columns: 7
Tuning Data Rows:    1517
Tuning Data Columns: 7
Label Column: deep
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    55128.53 MB
	Train Data (Original)  Memory Usage: 0.42 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Gen

In [15]:
predictor.leaderboard(train_data,silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist,0.997165,0.192075,0.007999,0.002000,0.005999,0.007999,0.002000,0.005999,1,True,2
1,ExtraTreesMSE,0.910686,0.373948,0.113264,0.031318,0.247931,0.113264,0.031318,0.247931,1,True,7
2,RandomForestMSE,0.909659,0.378249,0.125276,0.031335,0.429553,0.125276,0.031335,0.429553,1,True,5
3,WeightedEnsemble_L2,0.786689,0.395220,0.279544,0.070470,4.073525,0.003000,0.000000,0.112611,2,True,11
4,LightGBMLarge,0.681782,0.369663,0.007999,0.001999,0.757722,0.007999,0.001999,0.757722,1,True,10
5,LightGBM,0.565112,0.374504,0.006999,0.002067,0.530734,0.006999,0.002067,0.530734,1,True,4
6,CatBoost,0.563974,0.375674,0.009000,0.002750,2.304623,0.009000,0.002750,2.304623,1,True,6
7,XGBoost,0.519345,0.364562,0.022005,0.002999,0.448072,0.022005,0.002999,0.448072,1,True,8
8,KNeighborsUnif,0.478740,0.204346,0.009171,0.002001,0.726379,0.009171,0.002001,0.726379,1,True,1
9,LightGBMXT,0.469020,0.358349,0.028706,0.006000,0.953458,0.028706,0.006000,0.953458,1,True,3


In [16]:
predictor.leaderboard(test_data,silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.395220,0.395220,0.248187,0.070470,4.073525,0.003998,0.000000,0.112611,2,True,11
1,RandomForestMSE,0.378249,0.378249,0.114707,0.031335,0.429553,0.114707,0.031335,0.429553,1,True,5
2,CatBoost,0.375674,0.375674,0.009594,0.002750,2.304623,0.009594,0.002750,2.304623,1,True,6
3,LightGBM,0.374504,0.374504,0.004000,0.002067,0.530734,0.004000,0.002067,0.530734,1,True,4
4,ExtraTreesMSE,0.373948,0.373948,0.098725,0.031318,0.247931,0.098725,0.031318,0.247931,1,True,7
5,LightGBMLarge,0.369663,0.369663,0.005002,0.001999,0.757722,0.005002,0.001999,0.757722,1,True,10
6,XGBoost,0.364562,0.364562,0.017163,0.002999,0.448072,0.017163,0.002999,0.448072,1,True,8
7,LightGBMXT,0.358349,0.358349,0.012000,0.006000,0.953458,0.012000,0.006000,0.953458,1,True,3
8,NeuralNetTorch,0.264219,0.264219,0.009000,0.006000,12.945438,0.009000,0.006000,12.945438,1,True,9
9,KNeighborsUnif,0.204346,0.204346,0.003001,0.002001,0.726379,0.003001,0.002001,0.726379,1,True,1
